### MAKE DIFFERENT FEATURES AND MERGE ALL

- data aggregation
- join all data in loop (concat all all csv files with features)

In [ ]:
%load_ext autoreload
%autoreload 2

In [153]:
import os
import re
import dill
import sys
import pandas as pd
import numpy as np
import gc
import warnings

import matplotlib.pyplot as plt

from os import listdir
from dstools.spark import init_spark2

%matplotlib inline

In [3]:
spark = init_spark2({
    "appName": "ret_make_feats",
    "spark.yarn.queue": "default",
    "spark.driver.memory": "32g",
    "spark.executor.memoryOverhead": "2048",
    "spark.driver.extraJavaOptions": "-XX:ThreadStackSize=81920",
    "spark.executor.extraJavaOptions": "-XX:ThreadStackSize=81920",
    "spark.driver.maxResultSize": "45g",
    "spark.driver.memory": "45g"
})

In [4]:
sys.path.append('./scripts/')
from add_functions import catb_get_feature_imp, make_validation

-----

#### LOAD DATA

In [ ]:
data_path = '/user/kvliksak/retailhero'

In [ ]:
# read all data from spark

sdf_cliens = spark.read.csv(
    os.path.join(data_path, 'clients.csv'),
    inferSchema=True, header=True
)

sdf_products = spark.read.csv(
    os.path.join(data_path, 'products.csv'),
    inferSchema=True, header=True
)

sdf_purchase = spark.read.csv(
    os.path.join(data_path, 'purchases.csv'),
    inferSchema=True, header=True
)

sdf_train = spark.read.csv(
    os.path.join(data_path, 'uplift_train.csv'),
    inferSchema=True, header=True
)

sdf_test = spark.read.csv(
    os.path.join(data_path, 'uplift_test.csv'),
    inferSchema=True, header=True
)

In [ ]:
sdf_purchase = sdf_purchase\
    .join(
        sdf_products, 
        on=['product_id'], how='left'
    )
    
df_purchases = sdf_purchase.toPandas()

In [7]:
df_clients = sdf_cliens.toPandas()
df_train = sdf_train.toPandas()
df_test = sdf_test.toPandas()

df_clients = df_clients.set_index('client_id')
df_train = df_train.set_index('client_id')
df_test = df_test.set_index('client_id')

In [8]:
df_purchases.loc[:,'transaction_datetime'] =\
    pd.to_datetime(df_purchases['transaction_datetime'])
    
df_clients.loc[:,'first_issue_date'] =\
    pd.to_datetime(df_clients['first_issue_date'])
    
df_clients.loc[:,'first_redeem_date'] =\
    pd.to_datetime(df_clients['first_redeem_date'])

----

##### MAKE FEATURES FROM CLIENTS DATA

In [75]:
df_purchases.head(2)

,product_id,client_id,transaction_id,transaction_datetime,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,store_id,product_quantity,trn_sum_from_iss,trn_sum_from_red,level_1,level_2,level_3,level_4,segment_id,brand_id,vendor_id,netto,is_own_trademark,is_alcohol,max_date,min_date
0,9a80204f78,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,2.0,80.0,NaN,e344ab2e71,ed2ad1797c,b25baa9dd5,51647c28e9,116.0,082560ca58,63243765ed,0.031,0,0,2019-03-14 15:01:47,2018-12-01 07:12:45
1,da89ebd374,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,1.0,65.0,NaN,e344ab2e71,ed2ad1797c,0767853bf3,eaeb795060,14.0,cab440afaf,43acd80c1a,0.400,1,0,2019-03-14 15:01:47,2018-12-01 07:12:45


In [25]:
# GET MAX/MIN DATE per each client and leave only last CNT_LAST_DAYS days
CNT_LAST_DAYS = 30

df_purchases['max_date'] = df_purchases\
    .groupby(['client_id'])['transaction_datetime']\
    .transform(max)

df_purchases['min_date'] = df_purchases\
    .groupby(['client_id'])['transaction_datetime']\
    .transform(min)

In [57]:
cols = ['transaction_id', 'store_id', 'product_id', 'regular_points_received', 'level_1',
        'transaction_datetime', 'purchase_sum', 'product_quantity']

CNT_LAST_DAYS = 30

filter_cond =\
    ((df_purchases['max_date'] -\
      df_purchases['transaction_datetime']
     ).dt.days < CNT_LAST_DAYS)
    
df_purch_last = df_purchases\
    .set_index('client_id')[cols]

df_purch_last = df_purch_last[filter_cond.values]

In [74]:
# data perc selected
perc_last = np.round(
    df_purch_last.shape[0]/df_purchases.shape[0], 3)

perc_last

0.353

In [60]:
agg_cols_1 = ['mean', 'max', 'min', 'median']
agg_cols_2 = ['nunique']

df_purch_last_agg = df_purch_last\
    .reset_index()\
    .groupby('client_id')\
    .agg({
        'transaction_id': ['count'] + agg_cols_2,
        'product_id':agg_cols_2,
        'store_id': agg_cols_2,
        'level_1': agg_cols_2,
        'purchase_sum': agg_cols_1,
        'product_quantity': agg_cols_1,
        'regular_points_received': agg_cols_1
    })
    
df_purch_last_agg.columns =\
    ['__'.join(col).strip() + f'__last_{CNT_LAST_DAYS}'
     for col in df_purch_last_agg.columns.values]

In [70]:
# save:
df_purch_last_agg.to_csv(f'data/purch_last_{CNT_LAST_DAYS}_agg.csv')

------

In [76]:
### SAME FOR first N days:
cols = ['transaction_id', 'store_id', 'product_id', 'regular_points_received', 'level_1',
        'transaction_datetime', 'purchase_sum', 'product_quantity']

CNT_FIRST_DAYS = 30

filter_cond_first =\
    (( df_purchases['transaction_datetime'] - 
       df_purchases['min_date'] 
     ).dt.days < CNT_FIRST_DAYS)
    
    
df_purch_first = df_purchases\
    .set_index('client_id')[cols]

df_purch_first = df_purch_first[filter_cond_first.values]

In [77]:
# data perc selected
perc_last = np.round(
    df_purch_first.shape[0]/df_purchases.shape[0], 3)

perc_last

0.32

In [78]:
agg_cols_1 = ['mean', 'max', 'min', 'median']
agg_cols_2 = ['nunique']

df_purch_first_agg = df_purch_first\
    .reset_index()\
    .groupby('client_id')\
    .agg({
        'transaction_id': ['count'] + agg_cols_2,
        'product_id':agg_cols_2,
        'store_id': agg_cols_2,
        'level_1': agg_cols_2,
        'purchase_sum': agg_cols_1,
        'product_quantity': agg_cols_1,
        'regular_points_received': agg_cols_1
    })
    
df_purch_first_agg.columns =\
    ['__'.join(col).strip() + f'__first_{CNT_FIRST_DAYS}'
     for col in df_purch_first_agg.columns.values]

In [ ]:
# save:
df_purch_first_agg.to_csv(f'data/purch_first_{CNT_FIRST_DAYS}_agg.csv')

In [80]:
# clean
del df_purch_first, df_purch_last, df_purch_first_agg, df_purch_first_last
gc.collect();

----------

#### MAKE FEATURE FROM CLIENTS and CLIENTS + transaction dates

In [ ]:
df_clients['first_issue_date_weekday'] =\
    df_clients['first_issue_date'].dt.weekday.astype('object')
    
df_clients['first_redeem_date_weekday'] =\
    df_clients['first_redeem_date'].dt.weekday.astype('object')
    
df_clients['first_issue_date_hour'] =\
    df_clients['first_issue_date'].dt.hour.astype('object')
    
df_clients['first_redeem_date_hour'] =\
    df_clients['first_redeem_date'].dt.hour.astype('object')
    
df_clients['first_issue_date'] =\
    df_clients['first_issue_date'].astype(int)/10**9
    
df_clients['first_redeem_date'] =\
    df_clients['first_redeem_date'].astype(int)/10**9
    
df_clients['diff'] =\
    df_clients['first_redeem_date'] - df_clients['first_issue_date']

# set names
df_clients.columns =\
    ['cl_' + col for col in df_clients.columns.tolist()]

In [83]:
# FIND min - max transaction_datetime date for each client (client + store):

res_tmp = df_purchases\
    .assign(
        trans_date_int =\
        df_purchases['transaction_datetime'].astype(int)/10**9
    )\
    .groupby('client_id')\
    .agg({
        'trans_date_int': ['min','max', 'median', 'mean']
    })
    
res_tmp.columns =\
    ['__'.join(col).strip() 
     for col in res_tmp.columns.values]

res_tmp = res_tmp.assign(
    trans_dur = res_tmp['trans_date_int__max'] -\
                res_tmp['trans_date_int__min']
)

In [85]:
df_clients_full = pd.concat([
    df_clients,
    res_tmp 
], axis=1)

print(df_clients_full.shape)
df_clients_full.head(2)

(400162, 14)


,cl_first_issue_date,cl_first_redeem_date,cl_age,cl_gender,cl_first_issue_date_weekday,cl_first_redeem_date_weekday,cl_first_issue_date_hour,cl_first_redeem_date_hour,cl_diff,trans_date_int__min,trans_date_int__max,trans_date_int__median,trans_date_int__mean,trans_dur
client_id,,,,,,,,,,,,,,
000012768d,1.501948e+09,1.515094e+09,45,U,5,3,15,19,13146559.0,1.543648e+09,1.552576e+09,1.544951e+09,1.547536e+09,8927342.0
000036f903,1.491832e+09,1.492951e+09,72,F,0,6,13,12,1118613.0,1.543402e+09,1.552819e+09,1.548668e+09,1.548131e+09,9416461.0


In [89]:
df_clients_full = df_clients_full\
    .assign(
        diff__max_trans__first_issue =\
            df_clients_full['trans_date_int__max'] - df_clients_full['cl_first_issue_date'],
        diff__min_trans__first_issue =\
            df_clients_full['trans_date_int__min'] - df_clients_full['cl_first_issue_date'],
        diff__max_trans__first_redeem =\
             df_clients_full['trans_date_int__max'] - df_clients_full['cl_first_redeem_date'],
        diff__min_trans__first_redeem =\
             df_clients_full['trans_date_int__min'] - df_clients_full['cl_first_redeem_date']
    )

In [92]:
# save:
df_clients_full.to_csv('data/clients_and_trans.csv')

In [94]:
del df_clients_full
gc.collect();

---------

#### MAKE 2 step aggregation for purchase
- Step 1: groupby pair: ['client_id'] + cat_colum
- Step 2: simple aggregate by client

In [168]:
id_cols =  ['product_id', 'store_id', 'transaction_id', 'level_1', 'segment_id']
agg_cols_1 = ['mean', 'max', 'median']

agg_dfs = []

for id_col in id_cols:
    print(f'Id column: {id_col}')
    # step 1: agg by pair
    df_tmp = df_purchases\
        .groupby(['client_id', column])\
        .agg({
            'purchase_sum': agg_cols_1,
            'product_quantity': agg_cols_1,
            'trn_sum_from_iss': agg_cols_1,
            'trn_sum_from_red': agg_cols_1
        })

    df_tmp.columns =\
        ['__'.join(col).strip()
         for col in df_tmp.columns.values]

    df_tmp = df_tmp.reset_index()
    
    # step 2: agg by client:
    agg_feat_list = list(
        set(df_tmp.columns.tolist()) -\
        set(['client_id', 'store_id'])
    )

    agg_dict = {
        feat: ['mean', 'max'] for feat in agg_feat_list
    }

    df_tmp2 = df_tmp\
        .groupby(['client_id'])\
        .agg(agg_dict)

    df_tmp2.columns =\
        ['__'.join(col).strip() + f'__{id_col}'
         for col in df_tmp2.columns.values]
    
    print(f'Add new {df_tmp2.shape[1]} features \n')
    agg_dfs.append(df_tmp2)
   
    del df_tmp, df_tmp2

Id column: product_id
Add new 24 features 

Id column: store_id
Add new 24 features 

Id column: transaction_id
Add new 24 features 

Id column: level_1
Add new 24 features 

Id column: segment_id
Add new 24 features 



In [ ]:
# Concat all:
df_cont_total = pd.concat(
    agg_dfs, axis=1
)

print(df_cont_total.shape)
df_cont_total.head(2)
# (400162, 120)

In [169]:
# save:
df_cont_total.to_csv('data/purch_total_cont_agg.csv')

In [172]:
del agg_dfs, df_cont_total
gc.collect();

----

### MERGE ALL DATASETS:

In [173]:
mypath = '/data/dfs1/home/kvliksak/retail_hero/data'

all_features_files = list(
    set([f for f in listdir(mypath) if re.search(r'csv', f)]) -\
    set(['feats_baseline_train.csv', 'feats_baseline_test.csv'])
)

In [174]:
df_union = pd.DataFrame()

for file_ in all_features_files:
    print(f'File: {file_.split(".")[0]}')
    df_union = pd.concat([
        df_union, 
        pd.read_csv(
            os.path.join('data', file_), index_col=0)
        ], axis=1)

File: putch_total_agg_cat
File: putch_last_agg
File: putch_total_agg
File: putch_total_agg_date
File: purch_total_cont_agg
File: clients_and_trans
File: purch_last_30_agg


In [177]:
print(df_union.shape)
df_union.head(2)

(400162, 250)


,most_comm_product_id,nunique_product_id,most_comm_cnt_ratio_product_id,most_comm_sum_ratio_product_id,most_comm_transaction_id,nunique_transaction_id,most_comm_cnt_ratio_transaction_id,most_comm_sum_ratio_transaction_id,most_comm_store_id,nunique_store_id,most_comm_cnt_ratio_store_id,most_comm_sum_ratio_store_id,most_comm_level_1,nunique_level_1,most_comm_cnt_ratio_level_1,most_comm_sum_ratio_level_1,most_comm_level_2,nunique_level_2,most_comm_cnt_ratio_level_2,most_comm_sum_ratio_level_2,most_comm_level_3,nunique_level_3,most_comm_cnt_ratio_level_3,most_comm_sum_ratio_level_3,most_comm_level_4,nunique_level_4,most_comm_cnt_ratio_level_4,most_comm_sum_ratio_level_4,most_comm_segment_id,nunique_segment_id,most_comm_cnt_ratio_segment_id,most_comm_sum_ratio_segment_id,most_comm_vendor_id,nunique_vendor_id,most_comm_cnt_ratio_vendor_id,most_comm_sum_ratio_vendor_id,lats_total_count,lats_transaction_id__nunique,lats_regular_points_received__mean,lats_regular_points_received__max,lats_regular_points_received__min,lats_regular_points_received__median,lats_regular_points_spent__mean,lats_regular_points_spent__max,lats_regular_points_spent__min,lats_regular_points_spent__median,lats_purchase_sum__mean,lats_purchase_sum__max,lats_purchase_sum__min,lats_purchase_sum__median,lats_product_quantity__mean,lats_product_quantity__max,lats_product_quantity__min,lats_product_quantity__median,total_count,transaction_id__nunique,regular_points_received__mean,regular_points_received__max,regular_points_received__min,regular_points_received__median,express_points_received__mean,express_points_received__max,express_points_received__min,express_points_received__median,regular_points_spent__mean,regular_points_spent__max,regular_points_spent__min,regular_points_spent__median,express_points_spent__mean,express_points_spent__max,express_points_spent__min,express_points_spent__median,purchase_sum__mean,purchase_sum__max,purchase_sum__min,purchase_sum__median,product_quantity__mean,product_quantity__max,product_quantity__min,product_quantity__median,trn_sum_from_iss__mean,trn_sum_from_iss__max,trn_sum_from_iss__min,trn_sum_from_iss__median,trn_sum_from_red__mean,trn_sum_from_red__max,trn_sum_from_red__min,trn_sum_from_red__median,netto__mean,netto__max,netto__min,netto__median,min_date_trans,max_date_trans,days_from_last_trans,purchase_sum__median__mean,purchase_sum__median__max,product_quantity__mean__mean,product_quantity__mean__max,trn_sum_from_red__max__mean,trn_sum_from_red__max__max,trn_sum_from_iss__max__mean,trn_sum_from_iss__max__max,trn_sum_from_iss__median__mean,trn_sum_from_iss__median__max,product_quantity__max__mean,product_quantity__max__max,purchase_sum__max__mean,purchase_sum__max__max,product_quantity__median__mean,product_quantity__median__max,trn_sum_from_red__mean__mean,trn_sum_from_red__mean__max,purchase_sum__mean__mean,purchase_sum__mean__max,trn_sum_from_red__median__mean,trn_sum_from_red__median__max,trn_sum_from_iss__mean__mean,trn_sum_from_iss__mean__max,purchase_sum__median__mean.1,purchase_sum__median__max.1,product_quantity__mean__mean.1,product_quantity__mean__max.1,trn_sum_from_red__max__mean.1,trn_sum_from_red__max__max.1,trn_sum_from_iss__max__mean.1,trn_sum_from_iss__max__max.1,trn_sum_from_iss__median__mean.1,trn_sum_from_iss__median__max.1,product_quantity__max__mean.1,product_quantity__max__max.1,purchase_sum__max__mean.1,purchase_sum__max__max.1,product_quantity__median__mean.1,product_quantity__median__max.1,trn_sum_from_red__mean__mean.1,trn_sum_from_red__mean__max.1,purchase_sum__mean__mean.1,purchase_sum__mean__max.1,trn_sum_from_red__median__mean.1,trn_sum_from_red__median__max.1,trn_sum_from_iss__mean__mean.1,trn_sum_from_iss__mean__max.1,purchase_sum__median__mean.2,purchase_sum__median__max.2,product_quantity__mean__mean.2,product_quantity__mean__max.2,trn_sum_from_red__max__mean.2,trn_sum_from_red__max__max.2,trn_sum_from_iss__max__mean.2,trn_sum_from_iss__max__max.2,trn_sum_from_iss__median__mean.2,trn_sum

In [195]:
### SAVE FULL DATA:
df_union.to_csv('data/full_features_df.csv')

---

### NOTE:

In [191]:
# READ CLIENT CAT FEATS CORRECTLY:

cat_client_cols = [
    'cl_first_issue_date_weekday', 'cl_first_redeem_date_weekday',
    'cl_first_issue_date_hour', 'cl_first_redeem_date_hour'
]

cl_dtypes = {feat: 'object' for feat in cat_client_cols}
a = pd.read_csv('data/clients_and_trans.csv', dtype=cl_dtypes)